# Baby Cry Classifier using mL Algorithms

---

### Importing the needful Libraries

In [ ]:
# !conda install librosa -y

In [1]:
import pandas as pd
import numpy as np
import pickle 
import wave
import contextlib
import os
import math
import librosa 
from tqdm import tqdm

### Gather all the Sound (cry) datasets from the corresponding folders

In [2]:
#Store all audio files in dictionary where key: filename, value: label

raw_audio = dict()


directory = 'Food'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'hungry'
    else:
        continue

directory = 'Poop'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'diaper'
    else:
        continue
        
directory = 'Hug'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'hug'
    else:
        continue

In [3]:
raw_audio['Food/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu.wav']

'hungry'

In [4]:
def split_me(strng, sep, pos):
    strng = strng.split(sep)
    return sep.join(strng[:pos]), sep.join(strng[pos:])

In [5]:
def load_audio(filename, mono=False, fs=44100):
    """
    take a wav audio file and gives back:
    
    
    
    """
    file_base, file_extension = os.path.splitext(filename)
    if file_extension == '.wav':
        _audio_file = wave.open(filename)
        sample_rate = _audio_file.getframerate()
        sample_width = _audio_file.getsampwidth()
        number_of_channels = _audio_file.getnchannels()
        number_of_frames = _audio_file.getnframes()
        # Read raw bytes
        data = _audio_file.readframes(number_of_frames)
        _audio_file.close()

        # Convert bytes based on sample_width
        num_samples, remainder = divmod(len(data), sample_width * number_of_channels)
        if remainder > 0:
            raise ValueError('The length of data is not a multiple of sample size * number of channels.')
        if sample_width > 4:
            raise ValueError('Sample size cannot be bigger than 4 bytes.')

        if sample_width == 3:
            # 24 bit audio
            a = np.empty((num_samples, number_of_channels, 4), dtype=np.uint8)
            raw_bytes = np.fromstring(data, dtype=np.uint8)
            a[:, :, :sample_width] = raw_bytes.reshape(-1, number_of_channels, sample_width)
            a[:, :, sample_width:] = (a[:, :, sample_width - 1:sample_width] >> 7) * 255
            audio_data = a.view('<i4').reshape(a.shape[:-1]).T
        else:
            # 8 bit samples are stored as unsigned ints; others as signed ints.
            dt_char = 'u' if sample_width == 1 else 'i'
            a = np.fromstring(data, dtype='<%s%d' % (dt_char, sample_width))
            audio_data = a.reshape(-1, number_of_channels).T

        if mono:
            # Down-mix audio
            audio_data = np.mean(audio_data, axis=0)

        # Convert int values into float
        audio_data = audio_data / float(2 ** (sample_width * 8 - 1) + 1)

        # Resample
        if fs != sample_rate:
            audio_data = librosa.core.resample(audio_data, sample_rate, fs)
            sample_rate = fs

        return _audio_file, audio_data,sample_rate,sample_width,number_of_channels,number_of_frames
    return None, None

testing:

In [6]:
out_audio_file, out_audio_data,out_sample_rate,out_sample_width,out_number_of_channels,out_number_of_frames  = load_audio('Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu.wav')

print("out_audio_data: {0} \n \n out_sample_rate {1} \n \n out_sample_width: {2} \n \n out_number_of_channels: {3} \n \n out_number_of_frames:{4}".format(out_audio_data,out_sample_rate, out_sample_width, out_number_of_channels, out_number_of_frames))

out_audio_data: [[ 0.          0.          0.         ... -0.15944948 -0.16244011
  -0.1761726 ]
 [ 0.          0.          0.         ... -0.15944948 -0.16244011
  -0.1761726 ]] 
 
 out_sample_rate 44100 
 
 out_sample_width: 2 
 
 out_number_of_channels: 2 
 
 out_number_of_frames:302526


/var/folders/5j/22hm59fd3tgdwnxhbm26479r0000gn/T/ipykernel_21535/1887289389.py:36: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  a = np.fromstring(data, dtype='<%s%d' % (dt_char, sample_width))


In [6]:
def chop_cry(filename):
    handle = wave.open(filename)
    frame_rate = handle.getframerate()
    n_frames = handle.getnframes()
    window_size = 2 * frame_rate
    num_secs = int(math.ceil(n_frames/frame_rate))

    last_number_frames = 0
    #Slicing Audio file
    for i in range(num_secs):
        
        shortfilename = filename.split("/")[1].split(".")[0]
        #snippetfilename = 'audio/' + folder + '/' + shortfilename + 'snippet' + str(i+1) + '.wav'
        snippetfilename =  split_me(filename, '.',2)[0] + '-snippet' + str(i+1) + '.wav'
        print(snippetfilename)
        snippet = wave.open(snippetfilename ,'wb')
        snippet.setnchannels(2)
        snippet.setsampwidth(handle.getsampwidth())
        snippet.setframerate(frame_rate)
        #snippet.setsampwidth(2)
        #snippet.setframerate(11025)
        snippet.setnframes(handle.getnframes())
        
        snippet.writeframes(handle.readframes(window_size))
        handle.setpos(handle.tell() - 1 * frame_rate)
        #print snippetfilename, ":", snippet.getnchannels(), snippet.getframerate(), snippet.getnframes(), snippet.getsampwidth()
        
        #The last audio slice might be less than a second, if this is the case, we don't want to include it because it will not fit into our matrix 
        if last_number_frames < 1:
            last_number_frames = snippet.getnframes()
        elif snippet.getnframes() != last_number_frames:
            #print "this file doesnt have the same frame size!, remaming file"
            os.rename(snippetfilename, snippetfilename+".bak")
        snippet.close()

    handle.close()

In [50]:
# chop_song('Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu.wav')

Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet1.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet2.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet3.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet4.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet5.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet6.wav
Food_S/24f94b49-c8bc-4ed2-b43a-80b099f12936-1430534715095-1.7-m-04-hu-snippet7.wav


### Chop the audio file into 1 sec. snippets and save them in corresponding folders

In [7]:
for audio_file in raw_audio:
    try:
        chop_cry(audio_file)
#         out_audio_file, out_audio_data,out_sample_rate,out_sample_width,out_number_of_channels,out_number_of_frames =load_audio(audio_file)
#         print(out_number_of_frames)
    except Exception as arr:
        print(arr)
        pass
    

Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet1.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet2.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet3.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet4.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet5.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet6.wav
Food/8c66887c-8251-49fa-903b-c02d56405f58-1436856065644-1.7-m-26-hu-snippet7.wav
Food/d6727dfd-52a9-4daf-b423-e8ed1e8d65c8-1425413562261-1.2-m-72-hu-snippet1.wav
Food/d6727dfd-52a9-4daf-b423-e8ed1e8d65c8-1425413562261-1.2-m-72-hu-snippet2.wav
Food/d6727dfd-52a9-4daf-b423-e8ed1e8d65c8-1425413562261-1.2-m-72-hu-snippet3.wav
Food/d6727dfd-52a9-4daf-b423-e8ed1e8d65c8-1425413562261-1.2-m-72-hu-snippet4.wav
Food/d6727dfd-52a9-4daf-b423-e8ed1e8d65c8-1425413562261-1.2-m-72-hu-snippet5.wav
Food/d6727dfd-52a9-4daf-b423

Food/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430016779981-1.7-m-26-hu-snippet4.wav
Food/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430016779981-1.7-m-26-hu-snippet5.wav
Food/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430016779981-1.7-m-26-hu-snippet6.wav
Food/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430016779981-1.7-m-26-hu-snippet7.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet1.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet2.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet3.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet4.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet5.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet6.wav
Food/719bb382-a592-46b7-82d2-8b4a625263b7-1430376501744-1.7-m-48-hu-snippet7.wav
Food/a56e2494-d2b2-480e-af0b-de9ea8f33572-1436897467690-1.7-f-26-hu-snippet1.wav
Food/a56e2494-d2b2-480e-af0b

Food/92d08bbe-553f-4844-85b1-15d572f06ea2-1436873211119-1.7-m-04-hu-snippet7.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet1.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet2.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet3.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet4.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet5.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet6.wav
Food/0776b33b-c41a-4a2e-8b9c-f1c184be00c8-1437123838264-1.7-f-26-hu-snippet7.wav
Food/8b1faa77-e3ac-48af-bf2a-58aadebefcba-1436798541805-1.7-m-04-hu-snippet1.wav
Food/8b1faa77-e3ac-48af-bf2a-58aadebefcba-1436798541805-1.7-m-04-hu-snippet2.wav
Food/8b1faa77-e3ac-48af-bf2a-58aadebefcba-1436798541805-1.7-m-04-hu-snippet3.wav
Food/8b1faa77-e3ac-48af-bf2a-58aadebefcba-1436798541805-1.7-m-04-hu-snippet4.wav
Food/8b1faa77-e3ac-48af-bf2a

Food/9aa8bac5-eeb9-4f19-a4bf-7c439e87364b-1430745312460-1.7-m-04-hu-snippet3.wav
Food/9aa8bac5-eeb9-4f19-a4bf-7c439e87364b-1430745312460-1.7-m-04-hu-snippet4.wav
Food/9aa8bac5-eeb9-4f19-a4bf-7c439e87364b-1430745312460-1.7-m-04-hu-snippet5.wav
Food/9aa8bac5-eeb9-4f19-a4bf-7c439e87364b-1430745312460-1.7-m-04-hu-snippet6.wav
Food/9aa8bac5-eeb9-4f19-a4bf-7c439e87364b-1430745312460-1.7-m-04-hu-snippet7.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet1.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet2.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet3.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet4.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet5.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet6.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130702316-1.7-m-48-hu-snippet7.wav
Food/3d3e244d-20b1-4ec3-9732

Food/02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233150868-1.7-m-26-hu-snippet4.wav
Food/02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233150868-1.7-m-26-hu-snippet5.wav
Food/02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233150868-1.7-m-26-hu-snippet6.wav
Food/02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233150868-1.7-m-26-hu-snippet7.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet1.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet2.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet3.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet4.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet5.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet6.wav
Food/ed31b187-34af-4ff1-979c-3b8a5905fb7b-1430044853892-1.7-f-04-hu-snippet7.wav
Food/549a46d8-9c84-430e-ade8-97eae2bef787-1430130714659-1.7-m-48-hu-snippet1.wav
Food/549a46d8-9c84-430e-ade8

Food/d724fe27-4196-4029-b7fa-c2ad16b937d3-1437014249692-1.7-m-04-hu-snippet7.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet1.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet2.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet3.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet4.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet5.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet6.wav
Food/bb2bdafb-0477-4be4-8c41-1a6192e8c6d4-1430637640279-1.7-m-72-hu-snippet7.wav
Food/ca7d8d54-13cb-4060-8db4-90651b60c078-1434674769219-1.7-f-26-hu-snippet1.wav
Food/ca7d8d54-13cb-4060-8db4-90651b60c078-1434674769219-1.7-f-26-hu-snippet2.wav
Food/ca7d8d54-13cb-4060-8db4-90651b60c078-1434674769219-1.7-f-26-hu-snippet3.wav
Food/ca7d8d54-13cb-4060-8db4-90651b60c078-1434674769219-1.7-f-26-hu-snippet4.wav
Food/ca7d8d54-13cb-4060-8db4

### Transform .wav files to frequency spectrum "fingerprints" using MFCC algorithm and also tagging them

In [9]:

X = pd.DataFrame(columns = np.arange(45), dtype = 'float32').astype(np.float32)
j = 0
k = 0

print("Food folder begins...")
for i, filename in tqdm(enumerate(os.listdir('Food/'))):
    last_number_frames = -1
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("Food/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=1)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'hungry'
        X.loc[i] = x.loc[0]
        j = i 
print("Food folder done!")

print("Hug folder begins...")
for i, filename in tqdm(enumerate(os.listdir('Hug/'))):
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("Hug/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=1)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'hug'
        X.loc[i+j] = x.loc[0] 
        k = i         
print("Hug folder done!")


print("Poop folder begins...")   
for i, filename in tqdm(enumerate(os.listdir('Poop/'))):
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("Poop/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=1)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'diaper'
        X.loc[i+j+k] = x.loc[0]

print("Poop folder done!") 
#Do something with missing values. you might want to do something more sophisticated with missing values later
X = X.fillna(0)

Food folder begins...


2024it [27:59,  1.21it/s] 


Food folder done!
Hug folder begins...


129it [00:09, 14.00it/s]


Hug folder done!
Poop folder begins...


130it [00:09, 14.01it/s]

Poop folder done!


In [10]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
2,-361.221863,-362.912964,-366.202637,-367.268738,-386.205322,-402.769928,-339.163483,-257.949524,-208.454224,-186.770920,...,-239.881348,-277.235413,-345.489990,-372.664703,-363.644684,-366.289795,-349.077026,-326.958069,-327.089233,hungry
5,-322.297943,-316.886627,-320.066925,-315.647980,-304.988953,-295.689087,-292.120148,-284.999298,-281.294647,-277.429840,...,-259.501495,-258.044739,-251.927292,-247.636871,-256.006989,-260.140961,-250.572678,-233.324493,-212.506256,hungry
6,-328.540527,-321.409119,-313.513214,-298.519226,-290.412201,-292.358856,-288.474396,-241.152802,-185.098969,-165.315659,...,-245.268127,-251.418533,-248.377945,-240.206207,-240.528519,-236.388809,-230.413177,-240.351364,-269.093140,hungry
7,-780.018188,-775.185486,-775.426147,-760.823242,-749.060486,-742.104431,-741.585449,-720.068542,-664.467957,-624.855225,...,-803.661560,-805.518921,-801.402954,-803.255859,-805.827759,-792.891968,-790.274170,-771.678772,-737.214233,hungry
8,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,-590.431458,...,-475.234131,-486.563660,-501.200348,-511.803497,-512.426025,-504.738800,-499.761047,-505.097046,-498.655029,hungry


In [11]:
X.to_csv("NFCC_Result.csv",index=False)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
y = X[44]
del X[44]
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
#Code to hide deprication warnings

from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score


def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
        model = classifier(**kwargs)
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        return model.score(X_test, y_test), \
               precision_score(y_test, y_predict, average='micro'), \
               recall_score(y_test, y_predict, average='micro')

print("Model, Accuracy, Precision, Recall")
print("Random Forest:", get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test, n_estimators=25, max_features=5))
print("Logistic Regression:", get_scores(LogisticRegression, X_train, X_test, y_train, y_test))
print("Decision Tree:", get_scores(DecisionTreeClassifier, X_train, X_test, y_train, y_test))
print("SVM:", get_scores(SVC, X_train, X_test, y_train, y_test))

Model, Accuracy, Precision, Recall
Random Forest: (0.8787878787878788, 0.8787878787878788, 0.8787878787878788)
Logistic Regression: (0.8741258741258742, 0.8741258741258742, 0.8741258741258742)
Decision Tree: (0.8065268065268065, 0.8065268065268065, 0.8065268065268065)
SVM: (0.8741258741258742, 0.8741258741258742, 0.8741258741258742)


/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Persist the model in disk

In [27]:
def pickle_model(model, modelname):
    with open('models/' + str(modelname) + '.pkl', 'wb') as f:
        return pickle.dump(model, f)

In [28]:
model = RandomForestClassifier()
model.fit(X,y)
pickle_model(model, "myRandomForest")

In [29]:
model = LogisticRegression()
model.fit(X,y)
pickle_model(model, "myLogesticRegression")

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
model = DecisionTreeClassifier()
model.fit(X,y)
pickle_model(model, "myDecisionTreeClassifier")

In [31]:
model = SVC()
model.fit(X,y)
pickle_model(model, "mySVC")

### How to invoke perviously saved model

In [32]:
def getModel(pickle_path):
    with open(pickle_path, 'rb') as f:
        return cPickle.load(f)

## Eval

In [33]:
model = getModel("models/myRandomForest.pkl")

### make chop new audio reusable

In [47]:
def chop_new_audio(filename):
    handle = wave.open(filename)
    frame_rate = handle.getframerate()
    n_frames = handle.getnframes()
    window_size = 1 * frame_rate
    num_secs = int(math.ceil(n_frames/frame_rate))
    print(filename)
    last_number_frames = 0
    # Slicing Audio file
    for i in range(num_secs):
        
        shortfilename = filename.split("/")[1].split(".")[0]
        snippetfilename = split_me(filename, '.',2)[0] + '-snippet' + str(i+1) + '.wav'
        print(snippetfilename)
        snippet = wave.open(snippetfilename ,'wb')
        snippet.setnchannels(2)
        snippet.setsampwidth(handle.getsampwidth())
        snippet.setframerate(frame_rate)
        #snippet.setsampwidth(2)
        #snippet.setframerate(11025)
        snippet.setnframes(handle.getnframes())
        snippet.writeframes(handle.readframes(window_size))
        handle.setpos(handle.tell() - 1 * frame_rate)
        #print snippetfilename, ":", snippet.getnchannels(), snippet.getframerate(), snippet.getnframes(), snippet.getsampwidth()
        
        #The last audio slice might be less than a second, if this is the case, we don't want to include it because it will not fit into our matrix 
        if last_number_frames < 1:
            last_number_frames = snippet.getnframes()
        elif snippet.getnframes() != last_number_frames:
            #print "this file doesnt have the same frame size!, remaming file"
            os.rename(snippetfilename, snippetfilename+".bak")
        snippet.close()

    handle.close()

In [48]:
chop_new_audio('raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc.wav')

raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet1.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet2.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet3.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet4.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet5.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet6.wav
raw_audio/a40b9b8d-e4bc-457a-badf-f38bdbc0c358-1430017040912-1.7-m-26-dc-snippet7.wav


In [49]:
predictions = []
for i, filename in enumerate(os.listdir('raw_audio/')):
    last_number_frames = -1
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("raw_audio/"+filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=1)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        prediction = model.predict(fingerprint)
        print(prediction)
        predictions.append(prediction[0])

['diaper']
['diaper']
['diaper']
['diaper']
['diaper']
['diaper']
['diaper']


In [50]:
from collections import Counter
data = Counter(predictions)
print(data.most_common())   # Returns all unique items and their counts
print(data.most_common(1)) 

[('diaper', 7)]
[('diaper', 7)]
